# 6. Quiz¶
## 1. 과제 개요 교육생들은 OpenAI의 최신 API를 활용하여 주어진 텍스트 파일을 읽고, 내용을 요약한 후, 이를 음성 파일(MP3)로 변환하는 파이썬 프로그램을 개발해야 합니다. 본 과제를 통해 학생들은 다음과 같은 핵심 기술을 학습할 수 있습니다.
OpenAI API를 활용한 텍스트 요약 OpenAI의 텍스트 음성 변환(TTS) 기능 적용 Python 환경에서 파일 입출력 처리 python-dotenv를 활용한 API 키 관리

## 2. 요구 사항
a. 텍스트 파일 읽기: 프로그램은 지정된 텍스트 파일(ch06_quiz.txt)을 읽어와야 합니다. 
b. 텍스트 요약: OpenAI API를 사용하여 긴 텍스트를 30자 이내로 요약해야 합니다. 
c. 음성 변환: 요약된 내용을 OpenAI의 TTS(Text-to-Speech) API를 활용하여 음성데이터로 변환합니다. 
d. MP3 파일 저장: 생성된 음성 데이터를 ch06_quiz.mp3 파일로 저장해야 합니다. 
e. 환경변수 사용: API 키는 코드에 직접 포함하지 않고, .env 파일을 이용해 관리해야 합니다. 
f. 지정된 보이스 사용: OpenAI TTS에서 제공하는 특정 보이스("nova")를 사용해야 합니다.

## 3. 참고 자료
https://platform.openai.com/docs/guides/text-to-speech
https://platform.openai.com/docs/overview

In [1]:
# 필요한 라이브러리 임포트
from dotenv import load_dotenv
from openai import OpenAI
import os

# 환경변수 로드 및 OpenAI 클라이언트 초기화
load_dotenv()
client = OpenAI()

# 1. 텍스트 파일 읽기
def read_text_file(file_path):
    """지정된 경로의 텍스트 파일을 읽어 내용을 반환합니다."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        print(f"✅ 파일 읽기 완료: {file_path}")
        print(f"📄 원본 텍스트 길이: {len(content)}자")
        return content
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None

# 2. OpenAI API를 활용한 텍스트 요약 (70자 이내)
def summarize_text(text):
    """주어진 텍스트를 70자 이내로 요약합니다."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # 경제적인 모델 사용
            messages=[
                {"role": "system", "content": "당신은 텍스트를 간결하고 정확하게 요약하는 전문가입니다. 반드시 70자 이내로 요약해야 합니다."},
                {"role": "user", "content": f"다음 텍스트를 70자 이내로 요약해 주세요:\n\n{text}"}
            ],
            max_tokens=50  # 요약문 길이 제어
        )
        summary = response.choices[0].message.content.strip()
        print(f"✅ 텍스트 요약 완료")
        print(f"📝 요약문 ({len(summary)}자): {summary}")
        return summary
    except Exception as e:
        print(f"❌ 요약 중 오류 발생: {e}")
        return None

# 3. 요약된 텍스트를 음성으로 변환하고 MP3 파일로 저장
def text_to_speech_and_save(text, output_path):
    """주어진 텍스트를 OpenAI TTS API를 사용하여 음성으로 변환 후 MP3 파일로 저장합니다."""
    try:
        # TTS API 호출 (nova 목소리 사용)
        response = client.audio.speech.create(
            model="tts-1",
            voice="nova",
            input=text
        )
        
        # 음성 데이터 저장
        response.stream_to_file(output_path)
        
        # 파일 크기 확인
        file_size = os.path.getsize(output_path)
        print(f"✅ 음성 변환 및 저장 완료: {output_path}")
        print(f"🔊 파일 크기: {file_size:,} byte")
        return True
    except Exception as e:
        print(f"❌ 음성 변환 중 오류 발생: {e}")
        return False

# 메인 실행 부분
def main():
    # 파일 경로 설정
    input_file = 'data/ch06_quiz.txt'
    output_file = 'data/ch06_quiz.mp3'
    
    # 작업 시작 메시지
    print("="*60)
    print("🎯 OpenAI API 기반 텍스트 요약 및 음성 변환 프로그램 시작")
    print("="*60)
    
    # 1단계: 텍스트 파일 읽기
    original_text = read_text_file(input_file)
    if original_text is None:
        return
    
    # 2단계: 텍스트 요약
    summary_text = summarize_text(original_text)
    if summary_text is None:
        return
    
    # 3단계: 음성 변환 및 저장
    success = text_to_speech_and_save(summary_text, output_file)
    
    # 최종 완료 메시지
    print("="*60)
    if success:
        print("🎉 모든 작업이 성공적으로 완료되었습니다!")
        print(f"📂 생성된 파일: {output_file}")
    else:
        print("❌ 작업이 실패했습니다.")
    print("="*60)

# 프로그램 실행
if __name__ == "__main__":
    main()

🎯 OpenAI API 기반 텍스트 요약 및 음성 변환 프로그램 시작
✅ 파일 읽기 완료: data/ch06_quiz.txt
📄 원본 텍스트 길이: 400자
✅ 텍스트 요약 완료
📝 요약문 (42자): AI는 기계의 지능으로, NLP, 컴퓨터 비전, 로보틱스 등에서 활용됩니다.
✅ 음성 변환 및 저장 완료: data/ch06_quiz.mp3
🔊 파일 크기: 95,040 byte
🎉 모든 작업이 성공적으로 완료되었습니다!
📂 생성된 파일: data/ch06_quiz.mp3


C:\Users\Admin\AppData\Local\Temp\ipykernel_17816\2570749534.py:55: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(output_path)
